In [3]:
# Author: Alex Trostanovsky, 31-08-2018
#
# These notebooks contain the Exploratory Data Analysis I've conducted on Lending Club data
# Using third party data sources from:
# -  United States Department of Labour
# -  International Health Organization
#
# This notebook:
# 1)  preprocesses data from tables which contain:
#     -  number of employees, establishments, and cumulative annual pay per 3zip area (areas of US which are contained 
#        within a 3-digit zipcode prefix)
#     -  un/employment, level of education, demographic info per 3zip area
#     and merges these tables with lending club data to augment it
#
# 2) categorizes 'emp_title' data into discrete categories outlined by the International Labour Organization's (ILO) 
#    International Standard Classification of Occupations (ISCO)
#
# 3) splits into training/testing datasets, and trains a LightGBM classifier to categorize loan candidates who're are likely
#    to default (e.g. either 'Charged Off', 'Default', '30-120 Days Late)
#
# 4) produces cross-validation (AUCROC) metric result on the testing data-set with the trained model
#
# 5) generates a '_feature_importances' table for the trained model 

from contextlib               import contextmanager
from lightgbm                 import LGBMClassifier
from lightgbm                 import LGBMRegressor
from sklearn.metrics          import roc_auc_score, roc_curve
from sklearn.metrics          import mean_squared_error
from sklearn.model_selection  import KFold, StratifiedKFold
from sklearn.cross_validation import train_test_split
from math                     import sqrt
from tqdm                     import tqdm

import numpy as np
import pandas as pd
import gc
import time
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# convert percent string to float
def p2f(x):
    return float(x.strip('%'))/100

In [5]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [6]:
def preprocess_estabs():

    # read in dataframe containing establishment, employees, and annual payroll per zipcode 

    zips = pd.read_csv('../data/third_party/estab_emp_pay_by_3zip.csv')
    zips = zips.drop(columns = {'3zip.1'})
    zips.rename(columns = {'3zip': 'zip_code'}, inplace = True)

    # zfill columns with 3-digit zipcode prefix to contain 3 digits (i.e. '11' - '011')

    zips['zip_code'] = zips.zip_code.apply(lambda x: str(int(x)).zfill(3))
    return zips

In [7]:
def preprocess_lat_lons():
    # read in dataframe containing the median lat-lon per 3-digit prefix zipcode

    lat_lon = pd.read_csv('../data/third_party/prezip_to_lat_lon.csv')
    lat_lon.rename(columns = {'prezip': 'zip_code'}, inplace = True)
    lat_lon['zip_code'] = lat_lon['zip_code'].apply(str)
    return lat_lon

In [8]:
def preprocess_unemp():
    # read in dataframe containing unemployment data for zipcodes aggregated (mean) by their first 3 digits
    unemp = pd.read_csv('../data/third_party/zip_to_employment_stats.csv')
    unemp = unemp.drop(columns = {'prezip'})
    unemp['zip_code'] = unemp.zip_code.apply(lambda x: str(int(x)).zfill(3))
    return unemp

In [9]:
def preprocess_third_party_src():
    zips    = preprocess_estabs()
    lat_lon = preprocess_lat_lons()    
    unemp   = preprocess_unemp() 
    return zips, lat_lon, unemp

In [10]:
def letterSubGradesToNumeric(df):
    """Convert the letter+number sub_grade parameter in the Lending Club datatables to a numeric score.
    i.e.
    A1 = 36
    .
    .
    .
    G6 = 1
    
    Parameters
    ----------
    
    df (pd.Dataframe):
        the dataframe containing the lending club data
        
    Return
    ------
    
    df (pd.Dataframe):
        dataframe containing the numeric sub_grade score
    
    """
    grades = df.sub_grade.unique().tolist()
    gradeDict = {}
    for grade in grades:
        if type(grade) is str : 
            
            # subtract ascii value of 'A' first character in subgrade
            # multiply by 5 (there are 6 letters: A, B, C, D, E, F)
            # add the the second character in the sub_grade (the digit)
            num = (ord(grade[0]) - 65)*5 + int(grade[1]) 
            
            # want numeric grade to be ascending, so subtract from 36
            gradeDict[grade] = 36 - num 
    df['sub_grade'].replace(gradeDict, inplace = True)
    return df

In [11]:
def preprocess_LC_df(df):
    # drop unnecessary columns
    # id        = doesn't exist (id of loan - not provided by LC)
    # emp_title = too many entries ( > 30 000)
    # url       = url of user account at LC

    df.drop(columns = ['id', 'url', 'title'], inplace = True)

    df['int_rate']   = df['int_rate'].dropna().apply(p2f)
    df['int_rate']   = pd.to_numeric(df['int_rate'])
    df['revol_util'] = df['revol_util'].dropna().apply(p2f)
    df['revol_util'] = pd.to_numeric(df['revol_util'])

    # calculate feature - length of loan

    df['credit_length'] = df['loan_amnt'] / df['installment']
    # convert sub_grades to corresponding numeric value

    df = letterSubGradesToNumeric(df)

    df['zip_code'] = df['zip_code'].dropna().apply(lambda x: x.strip('xx'))
    
    # get only lower half of sub_grades ( 1 - 18, G6 - D3 )
    # not necessary for current analysis

    # df =  df[df['sub_grade'] <= 18]
    
    # merge dataframe containing establishment, employees, and annual payroll per zipcode 
    # and   dataframe containing the median lat-lon per 3-digit prefix zipcode
    # and   dataframe containing unemployment data
    # to    lending club dataframe  
    
    zips, lat_lon, unemp = preprocess_third_party_src()

    df = pd.merge(df, zips,    on = 'zip_code', how = 'left')
    print("merged establishments/employees/payroll")
    df = pd.merge(df, lat_lon, on = 'zip_code', how = 'left')
    print("merged lat/lons")
    df = pd.merge(df, unemp,   on = 'zip_code', how = 'left')
    print("merged unemployments")
    
    # generate a list of all features to be dropped (any feature which is more than 70% NULL)
    missing_fractions = df.isnull().mean().sort_values(ascending=False)
    drop_list = sorted(list(missing_fractions[missing_fractions > 0.3].index))
    df.drop(labels=drop_list, axis=1, inplace=True)

    # drop zip_code and addr_state categorical variables 
    # now that we have (approx.) median lat-lons per application

    df.drop(columns = ['zip_code', 'addr_state'], inplace = True)

    # make list of all time variables

    date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d', 'next_pymnt_d']

    # set start_date to construct timedeltas

    start_date = pd.Timestamp('2018-08-01')

    # change all string date features to pd.Timestamp objects

    for col in date_cols:
        if col in df.columns.tolist():
            df[col] = df[col].apply(pd.Timestamp) 
            print("converted " + str(col) + " to datetime")

    # calculate time deltas for each time features 

    for col in date_cols:
        if col in df.columns.tolist():
            df[col] = (df[col] - start_date)
            df[col] = df[col].dt.days

    return df

In [12]:
# read in saved csv

df = pd.read_csv('../data/exports/coded_emp_target.csv') 

# export adds redundant column for index values; get rid of that
df.drop(columns = ['Unnamed: 0'], inplace = True)

F:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (18,58,117,128,129,130,133,134,135,138,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
# drop rows which have digits for sub_mjr_grp_isco_08_title

df = df[df.sub_mjr_grp_isco_08_title.apply(lambda x: not x.isnumeric())]

In [14]:
# preprocess data 

df = preprocess_LC_df(df)

merged establishments/employees/payroll
merged lat/lons
merged unemployments
converted issue_d to datetime
converted earliest_cr_line to datetime
converted last_pymnt_d to datetime
converted last_credit_pull_d to datetime


In [15]:
# show all categorical columns and how many value are per columns before one hot encoding

df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

term                          2
grade                         7
emp_length                   11
home_ownership                4
verification_status           3
pymnt_plan                    2
purpose                      14
initial_list_status           2
application_type              2
hardship_flag                 2
disbursement_method           2
debt_settlement_flag          2
sub_mjr_grp_isco_08_title    30
dtype: int64

In [16]:
df, cols = one_hot_encoder(df)

# training in lightGBM doesn't allow usage of column names with: '[', ']', or, '<'
    
import re
regex = re.compile(r"\[|\]|<|\040|_\040|\(|\)", re.IGNORECASE)
df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<', '+', ' '))) else col for col in df.columns.values]

In [17]:
droplist = ['total_rec_prncp',         # principal recieved to date 
            'last_pymnt_amnt',         # last payment amount
            'last_pymnt_d',            # last payment day
            'total_rec_late_fee',      # late fees recieved to date
            'out_prncp',               # remaining outstanding principal for total amount funded
            'recoveries',              # post charge off gross recovery
            'total_rec_int',           # interest received to date
            'last_credit_pull_d',      # the most recent month LC pulled credit for this loan
            'total_pymnt',             # payments recieved to date for total amount funded
            'out_prncp_inv',           # remaining outstanding principal for portion of total amount funded by investors
            'total_pymnt_inv',         # payments received to date for portion of total amount funded by investors
            'last_fico_range_high',    # the upper boundary range the borrower’s last FICO pulled belongs to.
            'collection_recovery_fee', # post charge off collection fee
            'last_fico_range_low',     # the lower boundary range the borrower’s last FICO pulled belongs to.
            'issue_d']                 # the month which the loan was funded

In [18]:
# drop all columns indicative of defaulted loans
df.drop(columns = droplist, inplace = True)

In [19]:
df.to_csv('../data/exports/preproc_lc.csv')